In [137]:
%matplotlib inline

import pandas as pd
import numpy as np

from pandas.api.types import CategoricalDtype

from collections import defaultdict, Counter

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
import matplotlib.dates as mdates

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import patsy

from lifelines import KaplanMeierFitter

import dataStatsAnalysis as dsa
import dataStatsPlotting as dsp

dsp.SetParams()

In [138]:
def PowerMean(data, test_mean, alpha=0.05, alternative='two-sided', num_runs=100):
    a = np.array(data)
    
    # Set counter for significant p-values to zero
    count = 0
        
    # Run resampling of the array to simulate the alternative hypothesis of an existing difference from the test mean
    for i in range(num_runs):
        run_data = np.random.choice(a, size=len(a), replace=True)
            
        # Build a sampling distribution for the run
        mean_estimates = [np.random.choice(run_data, size=len(run_data), replace=True).mean() for _ in range(100)]
            
        # Create an rv of the results and calculate left and right side p-values
        rv = dsa.DiscreteRv(mean_estimates)
        p_value_right = 1 - rv.cdf(test_mean)
        p_value_left = rv.cdf(test_mean)
        
        # Case of a two-sided test
        if alternative == 'two-sided':
            if (p_value_right < alpha/2) or (p_value_left < alpha/2):
                count += 1
        
        # Case of testing for an effect that is smaller than the test mean
        elif alternative == 'smaller': 
            if p_value_right < alpha:
                count += 1
        
        # Case of testing for an effect that is larger than the test mean
        elif alternative == 'larger': 
            if p_value_left < alpha:
                count += 1
        
        else:
            raise ValueError("alternative has to be 'two-sided', 'smaller', or 'larger'")
            
    return count / num_runs

In [139]:
class UnimplementedMethodException(Exception):
    """Exception if someone calls a method that should be overridden."""

In [140]:
# Create the PowerTest superclass
class PowerTest():
    
    def __init__(self, data, test_stat, alpha=0.05, alternative='two-sided', num_runs=100):
        self.data = data
        self.test_stat = test_stat
        self.alpha = alpha
        self.alternative = alternative
        self.num_runs = num_runs
        self.PrepareData()
    
    # Provide functionality to convert the data into format needed for use in BuildRv
    # Ex. Convert to array, split data into component groups, etc.
    # See child classes for examples
    def PrepareData(self):
        UnimplementedMethodException()
    
    # Provide functionality to create a sampling distribution and build an rv
    # This involves doing one resample to simulate pulling an additional sample from the population,
    # then building building a sampling distribution for that sample,
    # and finally building an rv from the sampling distribution
    # See child classes for examples
    def BuildRv(self):
        UnimplementedMethodException()
    
    # Computes the pvalue of test stat from an rv,
    # and adds to pvalue_count if less than significance level
    def _RunPvalueCount(self):
        rv = self.BuildRv()
        p_value_right = 1 - rv.cdf(self.test_stat)
        p_value_left = rv.cdf(self.test_stat)
        
        # Case of a two-sided test
        if self.alternative == 'two-sided':
            if (p_value_right < self.alpha/2) or (p_value_left < self.alpha/2):
                self.pvalue_count+= 1
        
        # Case of testing for an effect that is smaller than the test mean
        elif self.alternative == 'smaller': 
            if p_value_right < self.alpha:
                self.pvalue_count += 1
        
        # Case of testing for an effect that is larger than the test mean
        elif self.alternative == 'larger': 
            if p_value_left < self.alpha:
                self.pvalue_count += 1
        
        else:
            raise ValueError("alternative has to be 'two-sided', 'smaller', or 'larger'")
    
    # Method for computing power 
    def Power(self):
        self.pvalue_count = 0
        for i in range(self.num_runs):
            self._RunPvalueCount()
            
        return self.pvalue_count / self.num_runs

In [141]:
class PTMean(PowerTest):
    
    def PrepareData(self):
        self.a = np.array(self.data)
    
    def BuildRv(self):
        run_data = np.random.choice(self.a, size=len(self.a), replace=True)
        mean_estimates = [np.random.choice(run_data, size=len(run_data), replace=True).mean() for _ in range(100)]
        rv = dsa.DiscreteRv(mean_estimates)
        return rv

In [142]:
mean_data = np.random.randint(-8,11,size=100)

In [143]:
powmean = PTMean(mean_data, 0, alternative='two-sided')

In [144]:
powmean.Power()

0.5

In [145]:
PowerMean(mean_data, 0, alternative='two-sided')

0.55

In [146]:
def PowerDiffMeans(a, b, test_diff_means, alpha = 0.05, alternative = 'two-sided', num_runs=100):
    a = np.array(a)
    b = np.array(b)
    
    # Set counter for significant p-values to zero
    count = 0
    
    # Run resampling of the arrays separately (not pooled) to simulate the alternative hypothesis of a difference existing
    for i in range(num_runs):
        sample1 = np.random.choice(a, size=len(a), replace=True)
        sample2 = np.random.choice(b, size=len(b), replace=True)
        
        diff_mean_results = []
        
        # Build a sampling distribution for the run
        for j in range(100):
            group1 = np.random.choice(sample1, size=len(sample1), replace=True)
            group2 = np.random.choice(sample2, size=len(sample2), replace=True)
            result = group1.mean() - group2.mean()
            diff_mean_results.append(result)
        
        # Create an rv of the results and calculate left and right side p-values
        rv = dsa.DiscreteRv(diff_mean_results)
        p_value_right = 1 - rv.cdf(test_diff_means)
        p_value_left = rv.cdf(test_diff_means)        

        # Case of a two-sided test
        if alternative == 'two-sided':
            if (p_value_right < alpha/2) or (p_value_left < alpha/2):
                count += 1
        
        # Case of testing for an effect that is smaller than the test difference of means
        elif alternative == 'smaller': 
            if p_value_right < alpha:
                count += 1
        
        # Case of testing for an effect that is larger than the test difference of means
        elif alternative == 'larger': 
            if p_value_left < alpha:
                count += 1
        
        else:
            raise ValueError("alternative has to be 'two-sided', 'smaller', or 'larger'")
    
    return count / num_runs

In [147]:
class PTDiffMeans(PowerTest):
    
    def PrepareData(self):
        self.a, self.b = self.data
        self.a = np.array(self.a)
        self.b = np.array(self.b)
    
    def BuildRv(self):
        # Create run data
        sample1 = np.random.choice(self.a, size=len(self.a), replace=True)
        sample2 = np.random.choice(self.b, size=len(self.b), replace=True)
        
        diff_mean_results = []
        
        # Build a sampling distribution for the run
        for j in range(100):
            group1 = np.random.choice(sample1, size=len(sample1), replace=True)
            group2 = np.random.choice(sample2, size=len(sample2), replace=True)
            result = group1.mean() - group2.mean()
            diff_mean_results.append(result)
        
        rv = dsa.DiscreteRv(diff_mean_results)
        
        return rv

In [148]:
import first

live, firsts, others = first.MakeFrames()

In [149]:
data = [firsts.prglngth.values, others.prglngth.values]

In [150]:
diffmeans = PTDiffMeans(data, 0)

In [151]:
# Seems to be working
diffmeans.Power()

0.27

In [152]:
PowerDiffMeans(firsts.prglngth.values, others.prglngth.values, 0)

0.21

In [153]:
len(firsts)

4413

In [154]:
# Gpower gives a power of 0.41, very close
np.mean(firsts.prglngth.values), np.mean(others.prglngth.values), np.std(firsts.prglngth.values), np.std(others.prglngth.values)

(38.60095173351461, 38.52291446673706, 2.791585069824391, 2.6155761106844744)

In [155]:
class PTCorrelation(PowerTest):
    def __init__(self, data, test_stat, alpha=0.05, alternative='two-sided', num_runs=100, method='pearson'):
        PowerTest.__init__(self, data, test_stat, alpha, alternative, num_runs)
        self.method = method
    
    def PrepareData(self):
        self.x, self.y = self.data
        self.df = pd.DataFrame({'x':self.x, 'y': self.y})
    
    def BuildRv(self):
        # Create run data
        run_data = self.df.sample(n=len(self.df), replace=True)
        
        corrs=[]
        
        # Build rv
        if self.method == 'pearson':          
            for _ in range(100):
                sample = run_data.sample(n=len(run_data), replace=True)
                r = stats.pearsonr(sample.x, sample.y)[0]
                corrs.append(r)
    
        elif self.method == 'spearman':  
            for _ in range(100):
                sample = run_data.sample(n=len(run_data), replace=True)
                r = stats.spearmanr(sample.x, sample.y)[0]
                corrs.append(r)
    
        else:
            raise Exception('Must enter either pearson or spearman as a string for method argument')
               
        rv = dsa.DiscreteRv(corrs)
        
        return rv

In [156]:
cleaned = live.dropna(subset=['agepreg', 'totalwgt_lb'])
data = cleaned.agepreg.values, cleaned.totalwgt_lb.values

In [157]:
powcorr = PTCorrelation(data,0)

In [158]:
powcorr.Power()

1.0

In [159]:
penguins = sns.load_dataset('penguins')

In [160]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [161]:
length = penguins.bill_length_mm.dropna()
depth = penguins.bill_depth_mm.dropna()

In [162]:
stats.pearsonr(length, depth)

(-0.23505287035553277, 1.1196621961373438e-05)

In [163]:
data = length.values, depth.values

In [164]:
powcorr_penguins = PTCorrelation(data,0)
powcorr_penguins.Power()

1.0

In [165]:
len(penguins.bill_length_mm.dropna()), len(penguins.bill_depth_mm.dropna())

(342, 342)

In [166]:
df_test = sns.load_dataset('diamonds')
df_test.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [167]:
df_test = df_test.dropna()

In [168]:
stats.pearsonr(df_test.depth, df_test.price)

(-0.010647404584142983, 0.013403249013409954)

In [169]:
data = df_test.depth.values, df_test.price.values

In [170]:
# Might be working, not sure how to confirm
# Comparison with Gpower results (z-tests - correlations: two independent Pearson's rs) is not close (0.41)
# Also strange that changing to smaller alternative didn't change the result of the calculation
testpowcorr = PTCorrelation(data,0, alternative='smaller')
testpowcorr.Power()

0.78

In [171]:
len(df_test.price.values)

53940

In [172]:
# Find a smaller correlation data set to test this with
a = np.random.randint(1,20,50)
a

array([ 5,  1,  2, 13, 17, 14, 15, 18, 15,  7,  1,  5,  3,  6,  5,  1, 14,
       12,  2, 18,  4,  9,  4, 14, 18,  9,  8, 17,  8, 15,  8,  7, 11, 10,
       10, 17, 14,  4,  7, 10,  9, 19, 18,  4,  8,  3,  7, 14,  6, 12])

In [173]:
b = np.random.randint(1,20,50)
b

array([13,  8, 15,  3, 14,  7,  9,  3,  5, 14,  5, 10, 19, 11, 17,  7, 13,
       14, 17,  1,  5,  7,  9,  8, 16,  6, 18, 10, 11, 12, 12,  8,  1, 17,
       19,  8,  7,  6, 17, 11,  8,  8,  2, 14, 18, 10, 10,  5, 16,  9])

In [174]:
stats.pearsonr(a,b)

(-0.3020885821758905, 0.03299429476207565)

In [175]:
data2 = a,b
testpowcorr2 = PTCorrelation(data2, 0, alternative='two-sided')
testpowcorr2.Power()

0.57

In [176]:
# A test with same data from Hypothesis test... notebook
c = [12,  7,  2, 12, 11,  5, 15,  5,  6, 12,  5, 15, 10,  2,  8,  2,  5,
       16, 16,  2, 17,  7, 11, 13, 13, 13, 17,  4,  9, 15, 13,  5,  5, 19,
        3,  4, 11, 16,  4, 14,  7, 12, 14,  9,  8,  4,  2,  8,  8,  8]
c = np.array(c)

In [177]:
d = [13,  3,  8, 18, 12,  8, 14, 19,  5,  2, 10, 17,  6, 12,  3,  2, 18,
        6, 11,  7, 12, 18, 16, 13, 13, 14, 19,  2, 14, 17,  3,  5, 14,  2,
       10, 14,  9, 11,  2,  5, 11, 18,  3,  4, 18,  1, 11,  4, 18,  3]
d= np.array(d)

In [178]:
# Yields same result so this doesn't appear to be a problem with using a class instead of a function
data3 = c,d
testpowcorr3 = PTCorrelation(data3, 0, alternative='two-sided')
testpowcorr3.Power()

0.22

In [179]:
# From a test of the two correlation hypothesis test methods (H0 and Ha) it seems there is quite a difference
# See test near bottom of Hypothesis test... notebook
# Try building a power test with the null hypothesis method

In [180]:
class PTCorrelationH0(PowerTest):
    def __init__(self, data, test_stat, alpha=0.05, alternative='two-sided', num_runs=100, method='pearson'):
        PowerTest.__init__(self, data, test_stat, alpha, alternative, num_runs)
        self.method = method
    
    def PrepareData(self):
        self.x, self.y = self.data
    
    def BuildRv(self):
        # Create run data
        run_x = np.random.permutation(self.x)
        run_y = self.y
        
        corrs=[]
        
        # Build rv
        if self.method == 'pearson':          
            for _ in range(100):
                x_perm = np.random.permutation(run_x)
                r = stats.pearsonr(x_perm , run_y)[0]
                corrs.append(r)
    
        elif self.method == 'spearman':  
            for _ in range(100):
                x_perm = np.random.permutation(run_x)
                r = stats.pearsonr(x_perm , run_y)[0]
                corrs.append(r)
    
        else:
            raise Exception('Must enter either pearson or spearman as a string for method argument')   
        
        rv = dsa.DiscreteRv(corrs)
        
        return rv

In [181]:
# Need the correlation from the orginal
# Interesting, the p-value from this is close to the one from the Ha computation not the H0 one
corr_original = stats.pearsonr(c,d)
corr_original

(0.1684736086385471, 0.24218627009242538)

In [182]:
# Is it working? gives almost zero power, sometimes 0.01
# Gpower gives 0.13
corr0 = PTCorrelationH0(data3, corr_original[0])
corr0.Power()

0.0

In [183]:
# Test original correlation hypothesis tests to see if there is a significant difference in pvalues

In [184]:
car = sns.load_dataset('car_crashes')
car.head(3)

,total,speeding,alcohol,not_distracted,no_previous,ins_premium,ins_losses,abbrev
0,18.8,7.332,5.640,18.048,15.040,784.55,145.08,AL
1,18.1,7.421,4.525,16.290,17.014,1053.48,133.93,AK
2,18.6,6.510,5.208,15.624,17.856,899.47,110.35,AZ


In [185]:
stats.pearsonr(car.no_previous, car.ins_premium)

(-0.1568952000433975, 0.2715478689798987)

In [193]:
# Checking pvalue from null hypothesis resampling
# It's very close to that of stats one above
resultH0 = dsa.ResampleCorrelation(car.no_previous.values, car.ins_premium.values)
dsa.PvalueFromEstimates(resultH0[1],resultH0[0])

0.25499999999999945

In [188]:
def ResampleCorrelation_Ha(x, y, iters=1000, method='pearson'):
    """Generates a correlation sampling distribution for the alternative hypothesis of correlation existing between the variables. 
    This is done by resampling x, y pairs and calculating correlation on new samples. 
    Can then make an rv of this distribution to calculate sampling distribution mean, std deviation (std error), and confidence interval (rv.interval). 
    Can also get a one-sided p-value for case of no difference null hypothesis using rv.cdf(0). 
    For two-sided p-value, can double the one-sided if sampling distribution is symmetrical or use the H0 version of this function. 
    Can also use the 'min' and 'max' built-ins to find what the most extreme values are from the simluations.

    Args:
        x (array-like): Input variable 1
        y (array-like): Input variable 2
        iters (int): The number of simulations to run (Defaults to 1000)
        method (string): Select 'pearson' or 'spearman' method (default: 'pearson')
        
    Returns:
        actual_r: Original actual correlation value
        corrs (array): Sampling distribution for the alternative hypothesis of no correlation obtained from resampling
    """
    if method == 'pearson':  
        # Calculate actual correlation
        actual_r = stats.pearsonr(x, y)[0]

        # Create a dataframe to hold the x and y values as pairs
        df = pd.DataFrame({'x':x, 'y': y})

        corrs=[]    
        for _ in range(iters):
            sample = df.sample(n=len(df), replace=True)
            r = stats.pearsonr(sample.x, sample.y)[0]
            corrs.append(r)
    
    elif method == 'spearman':
        # Calculate actual correlation
        actual_r = stats.spearmanr(x, y)[0]

        # Create a dataframe to hold the x and y values as pairs
        df = pd.DataFrame({'x':x, 'y': y})

        corrs=[]    
        for _ in range(iters):
            sample = df.sample(n=len(df), replace=True)
            r = stats.spearmanr(sample.x, sample.y)[0]
            corrs.append(r)
    
    else:
        raise Exception('Must enter either pearson or spearman as a string for method argument')
      
    return actual_r, np.array(corrs)


In [196]:
# Checking pvalue from alternative hypothesis resampling
# It's lower but still close to the stats one above
resultHa = ResampleCorrelation_Ha(car.no_previous, car.ins_premium)
dsa.PvalueFromEstimates(resultHa[1], 0)

0.17199999999999938

In [190]:
data = car.no_previous, car.ins_premium
car_power = PTCorrelation(data,0)
car_power.Power()

0.2

In [191]:
data = car.no_previous, car.ins_premium
car_powerH0 = PTCorrelationH0(data,-0.1568)
car_powerH0.Power()

0.0

In [ ]:
len(car)

In [ ]:
# G-power gives 0.19 for this

In [ ]:
# See how my power diff means one compares with GPower result
# Did this above and very similar result